In [2]:
import json
import pandas as pd
from flatten_json import flatten

In [3]:
# Foi consultado na base de dados SQL a quantidade máxima de cada evento,
# para posterior "nivelamento" dos dados
# Query: SELECT count(matches.id) FROM matches JOIN match_{EVENTO_NOME} ON (match_{EVENTO_NOME}.match_id = matches.id) GROUP BY matches.id ORDER BY count(matches.id) DESC;

MAX_AGAINSTGOALS = 2
MAX_GOALS = 8
MAX_PENALTYS = 3
MAX_RCARDS = 4
MAX_YCARDS = 11
MAX_SUBSTITUTIONS = 6

In [4]:
file = '../output/br-league.json'

def completeAnotherEvents(column, nmax):
    for i in range(len(column), nmax):
        column.append({
            "time": None,
            "team_id": None,
            "player_id": None
        })

def complete(_dict):    
    for d in _dict:
        for i in range(len(d["substitutions"]), MAX_SUBSTITUTIONS):
            d["substitutions"].append({
                "tactical_type": None,
                "id_player_in": None,
                "id_player_out": None,
                "time": None,
                "team_id": None
            })
        completeAnotherEvents(d["againstgoals"], MAX_AGAINSTGOALS)
        completeAnotherEvents(d["goals"],        MAX_GOALS)
        completeAnotherEvents(d["penaltys"],     MAX_PENALTYS)
        completeAnotherEvents(d["redcards"],     MAX_RCARDS)
        completeAnotherEvents(d["yellowcards"],  MAX_YCARDS)

def hasSubstitution(substitutions, time):
    for substitution in substitutions:
        if (time > substitution["time"]):
            return True
    return False
        
def goalsAfterSubs(time, substitutions, goals, N, team):
    for goal in goals:
        if (goal["team"] == team):
            if (hasSubstitution(substitutions, time) == True):
                break
            if (goal["time"] > time):
                N += 1
    return 1-(N*0.1)
        
def appendLabel(_dict, team):
    '''
        y(N): 1 - (N * 0,1), sendo N o número de gols sofridos após a substituição
    '''
    for d in _dict:
        N = 0
        for i in range(0, len(d["substitutions"])):
            if (d["substitutions"][i]["team"] == d[team]):
                time = d["substitutions"][i]["time"]
                N = goalsAfterSubs(time, d["substitutions"][i:len(d["substitutions"])], d["goals"], N, d[team])
        d["Y_{}".format(team)] = N
            
with open(file) as f:
    _dict = json.load(f)

# Adicionando o label Y, a qual deseja-se predizer
appendLabel(_dict, "home")
appendLabel(_dict, "away")
#print(_dict)
# Preencher dados faltantes
#complete(_dict)
#print(_dict)

In [5]:
dict_flattened = [flatten(d) for d in _dict]